### Reprolab example notebook
This is a notebook which you can use to try out ReproLab. 

Follow the tutorial in the README.md to get started. The reprolab demo will not work unless you run it from your home directory.

In [ ]:
!pip install matplotlib 

In [ ]:
# Get dummy datasets
from datetime import datetime
import random
from reprolab.experiment import persistio

datetime_of_data_retrieval = datetime.strptime('23/09/25 15:16:26', '%d/%m/%y %H:%M:%S')

def get_dummy_data(datetime):
    random_integers = [random.randint(0, 100) for _ in range(10)]
    correlated_integers = [x + random.randint(-50, 50) for x in random_integers]
    return random_integers, correlated_integers, datetime
    
'''
When a function with persistio decorator is run, it checks whether the function has been run previously.
If the function has been run previously, the same output will be given as the previous times the function was run.
Thus the result of the function will always be the same.

The persistio decorator helps ensure the reproducibility of data analysis 
by saving the exact dataset used at the time of an analysis. 
This is particularly useful when the data source is dynamic, 
like a continuously updated database without versioning or timestamps. 
By capturing and storing a static snapshot of the data, 
persistio allows you to re-run the analysis and get the same results, 
regardless of how the underlying database changes. 
'''
@persistio()
def retrieve_data_from_api(datetime):
    return get_dummy_data(datetime)

last_random_ints, last_correlated_ints, last_fetch_time = get_dummy_data(datetime.now())
cached_random_ints, cached_correlated_ints, cached_fetch_time = get_dummy_data(datetime_of_data_retrieval)

In [ ]:
# Print the data
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

def get_linear_fit(x_array, y_array):
    slope, y_intercept = np.polyfit(x_array, y_array, 1)
    return slope, y_intercept

def plot_linear_fit(x_array, y_array, color):
    slope, y_intercept = get_linear_fit(x_array, y_array)
    minimum_x = min(x_array)
    maximum_x = max(x_array)
    linear_fit_x = [minimum_x, maximum_x]
    linear_fit_y = [y_intercept + minimum_x*slope, y_intercept + maximum_x*slope]
    label = f"linear fit, y intercept = {y_intercept:.2f}, slope = {slope:.2f}"
    plt.plot(linear_fit_x, linear_fit_y, color, label=label)

def plot_data_and_linear_fit(x_array, y_array, datetime, color):
    plt.scatter(x_array, y_array, color=color, label=f"data from {datetime.strftime("%Y-%m-%d %H:%M:%S")}")
    plot_linear_fit(x_array, y_array, color)

plot_data_and_linear_fit(cached_random_ints, cached_correlated_ints, cached_fetch_time, "blue")
plot_data_and_linear_fit(last_random_ints, last_correlated_ints, last_fetch_time, "red")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.xlabel("random integers")
plt.ylabel("correlated integers")